# Assignment-9


### Import necessary libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import base64
import copy
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering



In [0]:
def splitTrainTest(data,percent):
    total=len(data)
    trainTotal=int(total*percent*0.01)
    testTotal=total-trainTotal
    return (data[0:trainTotal],data[trainTotal:total])

In [0]:
# Euclidean Distance
def distance(x,y,ax=1):
    return np.linalg.norm(x - y, axis=ax)
  
def MSE(xHat, x):
    return np.sum((xHat - x)**2) / len(x)

def MSE_prime(xHat, x):
    return xHat - x

### Layer Class
This class's object represent the layers in neural network. It stores the number of neurons in each layers, activations, activation function associated with each layer and their weight vector (initialize on gaussian distribute with mean =0 and std deviation=1).

In [0]:
class Layer:
    def __init__(self,nNodesCurrent, nNodesNext, activationF):
        self.nodesNo=nNodesCurrent
        self.activations = np.zeros([nNodesCurrent,1])
        self.activationF=activationF
        if nNodesNext==0:
            self.weights=None
        else:
            self.weights=np.random.normal(0, 1, size=(nNodesCurrent,nNodesNext))

### Neural Network Class
Class of neural network to perform autoencoding and decoding using mean square error between decoded input and actual input as cost function.
This class perform following tasks:
* Initialized the number of layers in neural network and create **Layer Class** object.
* Train the neural network on batches of inputs there by performing the forward and backward propogation using below helper methods.

#### Methods:
1. **Forward Propogation** : Perform the forward propogation, calculate and stores activations at each of the 
layer.
$$ z_1= w_1^TX $$
$$ a_1= f _1(z_1) $$
$$ z_2= w_2^Ta_1 $$
$$ \hat{y}= a_2= f_2(z_2) $$
2. **Calculate Error** : Here we calculate the mean square error of our neural network on the updated activations. This updation in activation take place after the updation of weights in gradient decent algorithm in backpropogation.
<br>
For multiclass classification we use the below cross entropy cost function:
$$ J =  \frac{1}{n} \sum\limits_{i}(\hat{x}-x)^2 $$
3. **Backward Propogation** : Here, we differentiate the cost function to minimize it, and find the optimal values of parameters ie weights at each of the layer.

<br>
also for cross entropy as a cost function and softmax as $f_2(z_2)$ , 
$$ \delta_3  =  \frac{\partial J}{\partial \hat{y} }  * \frac{\partial \hat{y}}{\partial z_2 } = (\hat{x} - x)*f^1 (z_2)  $$
<br>
$$ \therefore   \frac{\partial J}{\partial w_2 }= (\hat{x} - x) * a_1$$
<br><br>
Similarly to calculate parital derivative w.r.t weights of inner layers, we can use the chain rule
$$   \frac{\partial J}{\partial w_1 } = x^T *  \delta_3 *  w_2 *  f^1 (z_1)      $$

<hr>

**Some important activations functions and their derivatives used in our network**
**Softmax**
<br>
$$\sigma (z)_j = \frac{e^{z_j}}{\sum^K_{k=1} e^{z_j}}$$
**Sigmoid**
<br>
$$sigmoid(x) = \frac{1}{1+\epsilon ^ {-x}}$$
$$\frac{\partial sigmoid(x)}{\partial x} = sigmoid(x) * ( 1- sigmoid(x))$$

**Relu**
<br>
$$relu(x) = \max{(0,x)}$$
<br>
$$\frac{\partial relu(x)}{\partial x} = 1 \;\;\;\; if x>0  \\ 0 \;\;\; elsewhere$$

**tanh**
<br>
$$tanh(x) = \tanh{(x)}$$
<br>
$$\frac{\partial tanh(x)}{\partial x} = 1 - \tanh^2{(x)}$$

In [0]:
class NeuralNet:
    def __init__(self, totalLayers, noNodesList, activationFunctions):
        self.totalLayers=totalLayers
        self.noNodesList=noNodesList
        self.layers = []
        for i in range(totalLayers):
            currentLayerNodes=noNodesList[i]
            if i!=totalLayers-1:
                nextLayerNodes=noNodesList[i+1]
                ith_Layer=Layer(currentLayerNodes,nextLayerNodes,activationFunctions[i])
            else:
                ith_Layer=Layer(currentLayerNodes,0,activationFunctions[i])
            self.layers.append(ith_Layer)#append output layer as none

    def trainNetwork(self, data,outputLabels, batchSize, epochs, learningRate):
        self.learningRate=learningRate
        self.batchSize=batchSize;
     
        for x in range(epochs):
            i=0  
            while i<len(data):
                self.error=0
                self.forwardPropo(data[i:i+batchSize])#input
                self.calculateError(data[i:i+batchSize])#output
                self.backwardPropo(data[i:i+batchSize])
                i+=batchSize
            self.error /= batchSize
            print("Epoch ",x,"->Error: ", self.error)
          
    def forwardPropo(self, inputs):
        self.layers[0].activations =inputs
        for i in range(self.totalLayers-1):
            temp=np.matmul(self.layers[i].activations,self.layers[i].weights)  
            if self.layers[i+1].activationF == "sigmoid":
                self.layers[i+1].activations = self.sigmoid(temp)
            elif self.layers[i+1].activationF == "softmax":
                self.layers[i+1].activations = self.softmax(temp)
            elif self.layers[i+1].activationF == "relu":
                self.layers[i+1].activations = self.relu(temp)
            elif self.layers[i+1].activationF == "tanh":
                self.layers[i+1].activations = self.tanh(temp)
            elif self.layers[i+1].activationF == "linear":
                self.layers[i+1].activations = self.linear(temp)
            else:
                self.layers[i+1].activations = temp
        
    def calculateError(self,labels):
          self.error +=MSE(self.layers[self.totalLayers-1].activations,labels)
    
    def backwardPropo(self, labels):
        targets = labels
        i = self.totalLayers-1
        y = self.layers[i].activations
        
        delta=(y-targets)
        if self.layers[i].activationF=="linear":
            prime= self.linear_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
        elif self.layers[i].activationF=="sigmoid":
            prime= self.sigmoid_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
        elif self.layers[i].activationF=="relu":
            prime= self.relu_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
        elif self.layers[i].activationF=="tanh":
            prime= self.tanh_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
        delta=np.multiply(prime,delta)
        
        deltaw = np.dot(self.layers[i-1].activations.T, delta)/self.batchSize
        new_weights = self.layers[i-1].weights - self.learningRate * deltaw
        for i in range(i-1, 0, -1):
            if self.layers[i].activationF=="sigmoid":
                prime= self.sigmoid_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
            elif self.layers[i].activationF=="relu":
                prime= self.relu_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
            elif self.layers[i].activationF=="tanh":
                prime= self.tanh_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
            elif self.layers[i].activationF=="linear":
                prime= self.linear_derivative(np.matmul(self.layers[i-1].activations,self.layers[i-1].weights))
            
            delta=np.multiply(prime,delta.dot(self.layers[i].weights.T))
            deltaw = np.dot(self.layers[i-1].activations.T, delta)/self.batchSize

            self.layers[i].weights = new_weights
            new_weights = self.layers[i-1].weights - self.learningRate * deltaw
        self.layers[0].weights = new_weights
    
    
    def sigmoid(self, x):
        return np.divide(1, np.add(1, np.exp(np.negative(x))))
    
    def sigmoid_derivative(self,x):
        return (self.sigmoid(x)*(1-self.sigmoid(x)))
    
    def relu(self, x):
        return (x/700) * (x > 0)
    
    def relu_derivative(self,X):
        return 1. * (X > 0)
      
    def linear(self, x):
        return x/10
    
    def linear_derivative(self,X):
        return 1/10
    
    def softmax(self, x):
        x=np.nan_to_num(x)
        exp = np.exp(x)
        if isinstance(x[0], np.ndarray):
            return exp/np.sum(exp, axis=1, keepdims=True)
        else:
            return exp/np.sum(exp, keepdims=True)

    def tanh(self, x):
        return np.tanh(x)
    
    def tanh_derivative(self,x):
        return 1.0 - np.tanh(x) ** 2


## Question-1 problem of anomaly detection.

### Part-1 :  Dimenstionlity reduction using linear, non-linear activation function autoencoders , and deep autoencoders.

In [49]:
# data=pd.read_csv("data.csv").values
# X=data[:,0:29]
# Y=data[:,29]
# X = np.array(X, dtype=np.float64)
# X=(X-X.mean(axis=0))/(X.std(axis=0))

# numberofLayers=3
# batchSize=64
# epochs=50
# noofneurons=[29,14,29]#14 neurons in bottleneck layer

# activationFunctionsLinear=[None,"linear","linear"]
# learningRatLinear=0.1


# print("=================================================")
# print("Linear Activation function")
# print("=================================================")
# nnLinear=NeuralNet(numberofLayers,noofneurons,activationFunctionsLinear)
# nnLinear.trainNetwork(X,None,batchSize,epochs,learningRatLinear)
# nnLinear.batchSize = len(X)
# nnLinear.forwardPropo(X)
# encodedLinear = nnLinear.layers[int(nnLinear.totalLayers/2)].activations
# decodedLinear = nnLinear.layers[nnLinear.totalLayers-1].activations
# print("=================================================")
# print("Encoded input shape: ",X.shape)
# print("Encoded input shape: ",encodedLinear.shape)
# print("Reduced dimension from %d to %d "%(X.shape[1],encodedLinear.shape[1]))
# print("Decoded input shape: ",decodedLinear.shape)

# data=pd.read_csv("data.csv").values
# X=data[:,0:29]
# Y=data[:,29]
# X = np.array(X, dtype=np.float64)
# X=(X-X.mean(axis=0))/(X.std(axis=0))
# activationFunctionsNonLinear=[None,"relu","relu"]
# learningRatNonLinear=0.1

# print("=================================================")
# print("Non Linear Activation function (relu)")
# print("=================================================")
# nnNonLinear=NeuralNet(numberofLayers,noofneurons,activationFunctionsNonLinear)
# nnNonLinear.trainNetwork(X,None,batchSize,epochs,learningRatNonLinear)
# nnNonLinear.batchSize = len(X)
# nnNonLinear.forwardPropo(X)
# encodedNonLinear = nnNonLinear.layers[int(nnNonLinear.totalLayers/2)].activations
# decodedNonLinear = nnNonLinear.layers[nnNonLinear.totalLayers-1].activations
# print("=================================================")
# print("Encoded input shape: ",X.shape)
# print("Encoded input shape: ",encodedNonLinear.shape)
# print("Reduced dimension from %d to %d "%(X.shape[1],encodedNonLinear.shape[1]))
# print("Decoded input shape: ",decodedNonLinear.shape)


data=pd.read_csv("data.csv").values
X=data[:,0:29]
Y=data[:,29]
X = np.array(X, dtype=np.float64)
X=(X-X.mean(axis=0))/(X.std(axis=0))

numberofLayersDeep=7
noofneuronsDeep=[29,20,20,14,20,20,29]#14 neurons in bottleneck layer
activationFunctionsDeep=[None,"sigmoid","sigmoid","sigmoid","sigmoid","sigmoid","sigmoid"]
learningRatDeep=2

print("=================================================")
print("Deep autoencoders with non-linear activation functions")
print("=================================================")
deepNN=NeuralNet(numberofLayersDeep,noofneuronsDeep,activationFunctionsDeep)
deepNN.trainNetwork(X,None,batchSize,epochs,learningRatDeep)
deepNN.batchSize = len(X)
deepNN.forwardPropo(X)
encodedDeep = deepNN.layers[int(deepNN.totalLayers/2)].activations
decodedDeep = deepNN.layers[deepNN.totalLayers-1].activations
print("=================================================")
print("Encoded input shape: ",X.shape)
print("Encoded input shape: ",encodedDeep.shape)
print("Reduced dimension from %d to %d "%(X.shape[1],encodedDeep.shape[1]))
print("Decoded input shape: ",decodedDeep.shape)


Deep autoencoders with non-linear activation functions
Epoch  0 ->Error:  0.520476039484272
Epoch  1 ->Error:  0.49335064572938814
Epoch  2 ->Error:  0.4820249853805794
Epoch  3 ->Error:  0.4758653291187544
Epoch  4 ->Error:  0.46476110156170247
Epoch  5 ->Error:  0.45516778517885126
Epoch  6 ->Error:  0.4540339535763093
Epoch  7 ->Error:  0.4537308999597737
Epoch  8 ->Error:  0.4532106386831645
Epoch  9 ->Error:  0.45494557186318846
Epoch  10 ->Error:  0.45516853825639514
Epoch  11 ->Error:  0.4719212968996312
Epoch  12 ->Error:  0.4529024986060632
Epoch  13 ->Error:  0.4515554231583348
Epoch  14 ->Error:  0.45152449955445756
Epoch  15 ->Error:  0.4512254940999185
Epoch  16 ->Error:  0.45118123978346386
Epoch  17 ->Error:  0.4510218817371911
Epoch  18 ->Error:  0.4514499629135433
Epoch  19 ->Error:  0.4509456938121675
Epoch  20 ->Error:  0.4505831944124509
Epoch  21 ->Error:  0.45219659326057204
Epoch  22 ->Error:  0.4510971946880303
Epoch  23 ->Error:  0.4503992051255105
Epoch  24 ->

In [0]:
def KMeans(k,noofneurons=[29,14,29],activationFunctions=[None,"linear","linear"],learningRat=0.1,forgraph=False):
    data=pd.read_csv("data.csv").values
    X=data[:,0:29]
    Y=data[:,29]
    X = np.array(X, dtype=np.float64)
    X=(X-X.mean(axis=0))/(X.std(axis=0))
    
    numberofLayers=len(noofneurons)
    batchSize=64
    epochs=50
    
    
    nn=NeuralNet(numberofLayers,noofneurons,activationFunctions)
    nn.trainNetwork(X,None,batchSize,epochs,learningRat)
    nn.batchSize = len(X)
    nn.forwardPropo(X)
    
    
    X=nn.layers[nn.totalLayers-2].activations
    X=(X-X.mean(axis=0))/(X.std(axis=0))
    dimensions=X.shape[1]
    
    count=0
    cc=[]
    for i in range(k):
        centroid=np.random.random_sample(dimensions)
        cc.append(centroid)
    newCentroids=np.array(cc)
    oldCentroids=np.zeros(newCentroids.shape)
    clusterMap=np.zeros(len(X))
    error=distance(newCentroids,oldCentroids,None)

    count=0
    clusterPoints={}
    while error>0.01:

        for i in range(len(X)):
            dist=distance(X[i],newCentroids)
            clusterMap[i]=np.argmin(dist)
        oldCentroids=copy.deepcopy(newCentroids)
    # Finding the new centroids by taking the average value
        for i in range(k):
            points=[]
            indexes=[]
            for j in range(len(X)):
                if clusterMap[j]==i:
                    points.append(X[j])
                    indexes.append(j)
                    
            if len(points)>0:
                newCentroids[i]=np.mean(points,axis=0)
            clusterPoints[i]=indexes
        error=distance(newCentroids,oldCentroids,None)
        count+=1
    methodPurity=0
    graphLabels=[]
    graphClusters=[]
    for key in clusterPoints.keys():
        indexes=clusterPoints[key]
        labels=np.array([ Y[index] for index in indexes])
        uniq,counts=np.unique(labels,return_counts=True)
        countIndex=np.argmax(counts)
        methodPurity+=counts[countIndex]
        if forgraph:
          graphLabels.append(uniq[countIndex]+" : "+str(counts[countIndex]/len(indexes)))
          graphClusters.append(len(indexes))
          continue
        print("========================================")
        print("Cluster: ",key+1)
        print("========================================")
        print("%10s || %2s "%("class","count"))
        print("________________________")
        for a,b in zip(uniq,counts):
          print("%10s || %2s "%(a,b))
        print()
        print("Max class '%s' with purity %s . "%(uniq[countIndex],counts[countIndex]/len(indexes)))
        print("========================================")
    if forgraph:
      return (methodPurity/len(X),graphLabels,graphClusters)
 
    print("Method clustering Purity= ",methodPurity/len(X))

### Part-2 : K-means Clustering on  dimensionality reduced inputs.

#### Part-2-a : K-means with linear activation functions in autoencoder

In [56]:
KMeans(5,noofneurons=[29,14,29],activationFunctions=[None,"linear","linear"])

Epoch  0 ->Error:  0.4073320111139851
Epoch  1 ->Error:  0.3195436924633994
Epoch  2 ->Error:  0.28554317193374085
Epoch  3 ->Error:  0.25820880363823756
Epoch  4 ->Error:  0.232837025046499
Epoch  5 ->Error:  0.20830111051418027
Epoch  6 ->Error:  0.18161077541162954
Epoch  7 ->Error:  0.15212982922142956
Epoch  8 ->Error:  0.12310397671942745
Epoch  9 ->Error:  0.09913626211131475
Epoch  10 ->Error:  0.08276025739043531
Epoch  11 ->Error:  0.07338206143453922
Epoch  12 ->Error:  0.06872861198099695
Epoch  13 ->Error:  0.06664156615595504
Epoch  14 ->Error:  0.06576948418360085
Epoch  15 ->Error:  0.06543528356638965
Epoch  16 ->Error:  0.0653383256714521
Epoch  17 ->Error:  0.06535225328963787
Epoch  18 ->Error:  0.06542415162162458
Epoch  19 ->Error:  0.0655306817871712
Epoch  20 ->Error:  0.0656602280563564
Epoch  21 ->Error:  0.06580588419390279
Epoch  22 ->Error:  0.06596276419104381
Epoch  23 ->Error:  0.06612699453793146
Epoch  24 ->Error:  0.06629534311274678
Epoch  25 ->Error

#### Part-2-b : K-means with non linear activation functions (ReLu) in autoencoder

In [59]:
KMeans(5,noofneurons=[29,14,29],activationFunctions=[None,"relu","relu"])

Epoch  0 ->Error:  0.513435119380447
Epoch  1 ->Error:  0.4495585508430057
Epoch  2 ->Error:  0.4089321341331035
Epoch  3 ->Error:  0.29109174004438326
Epoch  4 ->Error:  0.23885953251159803
Epoch  5 ->Error:  0.23574014867510085
Epoch  6 ->Error:  0.23331456381908763
Epoch  7 ->Error:  0.23405574584307504
Epoch  8 ->Error:  0.22650590789338404
Epoch  9 ->Error:  0.21676345756128595
Epoch  10 ->Error:  0.20310200680344878
Epoch  11 ->Error:  0.18090896235162987
Epoch  12 ->Error:  0.16277660560699356
Epoch  13 ->Error:  0.14957211267835682
Epoch  14 ->Error:  0.14085934381439663
Epoch  15 ->Error:  0.13497765171191706
Epoch  16 ->Error:  0.1308557196678062
Epoch  17 ->Error:  0.12801726039279485
Epoch  18 ->Error:  0.12585806694160231
Epoch  19 ->Error:  0.125509587531424
Epoch  20 ->Error:  0.12194476015450402
Epoch  21 ->Error:  0.11998364280599307
Epoch  22 ->Error:  0.1184422041155556
Epoch  23 ->Error:  0.11724927958115422
Epoch  24 ->Error:  0.11642006655601009
Epoch  25 ->Error:

#### Part-2-c : K-means with deep autoencoder

In [69]:
KMeans(k=5,noofneurons=[29,20,20,14,20,20,29],activationFunctions=[None,"sigmoid","sigmoid","sigmoid","sigmoid","sigmoid","sigmoid"],learningRat=2)

Epoch  0 ->Error:  0.505281204562985
Epoch  1 ->Error:  0.5022032107654857
Epoch  2 ->Error:  0.5000170023420896
Epoch  3 ->Error:  0.4990086799722499
Epoch  4 ->Error:  0.511031071049781
Epoch  5 ->Error:  0.49630884612064013
Epoch  6 ->Error:  0.4942148779222599
Epoch  7 ->Error:  0.4941859967294104
Epoch  8 ->Error:  0.4937106738796286
Epoch  9 ->Error:  0.49321836080994264
Epoch  10 ->Error:  0.49611119262363235
Epoch  11 ->Error:  0.4940839155943787
Epoch  12 ->Error:  0.493293312617902
Epoch  13 ->Error:  0.4932302310582355
Epoch  14 ->Error:  0.493388599773983
Epoch  15 ->Error:  0.48869805558817836
Epoch  16 ->Error:  0.49170751056261075
Epoch  17 ->Error:  0.4909097308347008
Epoch  18 ->Error:  0.4870868245651469
Epoch  19 ->Error:  0.49062981575975284
Epoch  20 ->Error:  0.48868411515323684
Epoch  21 ->Error:  0.4876448792025728
Epoch  22 ->Error:  0.4879137615973732
Epoch  23 ->Error:  0.48698362493159336
Epoch  24 ->Error:  0.46874464535313


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in exp


Epoch  25 ->Error:  0.4675559008787901
Epoch  26 ->Error:  0.4669368769063373
Epoch  27 ->Error:  0.4697715786800912
Epoch  28 ->Error:  0.4688738593375014
Epoch  29 ->Error:  0.5041122700477888
Epoch  30 ->Error:  0.47677548423446936
Epoch  31 ->Error:  0.47385377377540044
Epoch  32 ->Error:  0.4652559400626735
Epoch  33 ->Error:  0.4576332305936404
Epoch  34 ->Error:  0.4578648256902832
Epoch  35 ->Error:  0.4572881208450969
Epoch  36 ->Error:  0.4568064891457092
Epoch  37 ->Error:  0.4569449016956713
Epoch  38 ->Error:  0.45610563448027736
Epoch  39 ->Error:  0.45606849136042077
Epoch  40 ->Error:  0.4564346682681912
Epoch  41 ->Error:  0.4559297387333969
Epoch  42 ->Error:  0.4553296002766752
Epoch  43 ->Error:  0.45505774104845864
Epoch  44 ->Error:  0.4550834601237778
Epoch  45 ->Error:  0.4549728282894243
Epoch  46 ->Error:  0.45495049794900544
Epoch  47 ->Error:  0.4549794381807294
Epoch  48 ->Error:  0.4550610832787538
Epoch  49 ->Error:  0.4549297816661503
Cluster:  1
     cl

In [0]:
def gMM(k,noofneurons=[29,14,29],activationFunctions=[None,"linear","linear"],learningRat=0.1,forgraph=False):
    data=pd.read_csv("data.csv").values
    X=data[:,0:29]
    Y=data[:,29]
    
    X = np.array(X, dtype=np.float64)
    X=(X-X.mean(axis=0))/(X.std(axis=0))
    
    numberofLayers=len(noofneurons)
    batchSize=64
    epochs=50
    
    nn=NeuralNet(numberofLayers,noofneurons,activationFunctions)
    nn.trainNetwork(X,None,batchSize,epochs,learningRat)
    nn.batchSize = len(X)
    nn.forwardPropo(X)
    
    dimensions=14
    X=nn.layers[nn.totalLayers-2].activations
#     Y=data[2]
    gmm = GaussianMixture(n_components = k)
    gmm.fit(X)
    labels=gmm.predict(X) 
    labelsDict={}
    for index,label in enumerate(labels):
        labelsDict.setdefault(label,[]).append(index)
    methodPurity=0
    graphLabels=[]
    graphClusters=[]
    for key in labelsDict.keys():
        indexes=labelsDict[key]
        labelss=np.array([Y[index] for index in indexes])
        uniq,counts=np.unique(labelss,return_counts=True)
        countIndex=np.argmax(counts)
        methodPurity+=counts[countIndex]
        if forgraph:
          graphLabels.append(uniq[countIndex]+" : "+str(counts[countIndex]/len(indexes)))
          graphClusters.append(len(indexes))
          continue
        print("========================================")
        print("Cluster: ",key+1)
        print("========================================")
        print("%10s || %2s "%("class","count"))
        print("________________________")
        for a,b in zip(uniq,counts):
          print("%10s || %2s "%(a,b))
        print()
        print("Max class '%s' with purity %s . "%(uniq[countIndex],counts[countIndex]/len(indexes)))       
        print("========================================")
    if forgraph:
      return (methodPurity/len(X),graphLabels,graphClusters)
    print("Method clustering Purity= ",methodPurity/len(X))

### Part-3 :Gaussian Mixture Model  on  dimensionality reduced inputs.

#### Part-3-a :GMM with linear activation functions in autoencoder

In [76]:
gMM(5,noofneurons=[29,14,29],activationFunctions=[None,"linear","linear"])

Epoch  0 ->Error:  0.37882112772925053
Epoch  1 ->Error:  0.25884593249867155
Epoch  2 ->Error:  0.19346516940503056
Epoch  3 ->Error:  0.14688304717684242
Epoch  4 ->Error:  0.11599182354744744
Epoch  5 ->Error:  0.09624900516222214
Epoch  6 ->Error:  0.08318023332118096
Epoch  7 ->Error:  0.07443386827693187
Epoch  8 ->Error:  0.06880141343978632
Epoch  9 ->Error:  0.06533167877304019
Epoch  10 ->Error:  0.0630827824512816
Epoch  11 ->Error:  0.061340634455753336
Epoch  12 ->Error:  0.05990490599966166
Epoch  13 ->Error:  0.05895258482855109
Epoch  14 ->Error:  0.05861719360867764
Epoch  15 ->Error:  0.058795486103554395
Epoch  16 ->Error:  0.05927114631182714
Epoch  17 ->Error:  0.059865783746777416
Epoch  18 ->Error:  0.06047872932389795
Epoch  19 ->Error:  0.06106560403675792
Epoch  20 ->Error:  0.06161144672440179
Epoch  21 ->Error:  0.06211410317056674
Epoch  22 ->Error:  0.06257615393042756
Epoch  23 ->Error:  0.06300147222862829
Epoch  24 ->Error:  0.0633939123562374
Epoch  25

#### Part-3-b : GMM with non linear activation functions (ReLu) in autoencoder

In [78]:
gMM(5,noofneurons=[29,14,29],activationFunctions=[None,"relu","relu"])

Epoch  0 ->Error:  0.5421247506956544
Epoch  1 ->Error:  0.4824182404398691
Epoch  2 ->Error:  0.40367778005176014
Epoch  3 ->Error:  0.32653942699129673
Epoch  4 ->Error:  0.1767762107711481
Epoch  5 ->Error:  0.14029734329451193
Epoch  6 ->Error:  0.13908878676024192
Epoch  7 ->Error:  0.13955883678603098
Epoch  8 ->Error:  0.14179366085636597
Epoch  9 ->Error:  0.14629339444540398
Epoch  10 ->Error:  0.15274217958692243
Epoch  11 ->Error:  0.15188333167168694
Epoch  12 ->Error:  0.14141569362628018
Epoch  13 ->Error:  0.12881510003945323
Epoch  14 ->Error:  0.12473570542331665
Epoch  15 ->Error:  0.12441202597072913
Epoch  16 ->Error:  0.12519518604470684
Epoch  17 ->Error:  0.12452571950774909
Epoch  18 ->Error:  0.12444761279718504
Epoch  19 ->Error:  0.1233925756893049
Epoch  20 ->Error:  0.12293800204512113
Epoch  21 ->Error:  0.12221532653842387
Epoch  22 ->Error:  0.12183031311661097
Epoch  23 ->Error:  0.12156169340095568
Epoch  24 ->Error:  0.12121396285551377
Epoch  25 ->Er

#### Part-3-c : GMM with deep autoencoders

In [79]:
gMM(k=5,noofneurons=[29,20,20,14,20,20,29],activationFunctions=[None,"sigmoid","sigmoid","sigmoid","sigmoid","sigmoid","sigmoid"],learningRat=2)

Epoch  0 ->Error:  0.530106577993845
Epoch  1 ->Error:  0.49995459743965426
Epoch  2 ->Error:  0.49100899363872325
Epoch  3 ->Error:  0.4759720523826284
Epoch  4 ->Error:  0.47301201718523017
Epoch  5 ->Error:  0.4713688679608154
Epoch  6 ->Error:  0.4697816667397126
Epoch  7 ->Error:  0.467596046813513
Epoch  8 ->Error:  0.4660575186122525
Epoch  9 ->Error:  0.4649901692524672
Epoch  10 ->Error:  0.4634843703584394
Epoch  11 ->Error:  0.46310321490997436
Epoch  12 ->Error:  0.46256560580257294
Epoch  13 ->Error:  0.46225113322686157
Epoch  14 ->Error:  0.46225577274994506
Epoch  15 ->Error:  0.46170039564669496
Epoch  16 ->Error:  0.46147084597220706
Epoch  17 ->Error:  0.4652930820508521
Epoch  18 ->Error:  0.4612256126316757
Epoch  19 ->Error:  0.4612066190674667
Epoch  20 ->Error:  0.46272092807951104
Epoch  21 ->Error:  0.46278127447148426
Epoch  22 ->Error:  0.4608186635735086
Epoch  23 ->Error:  0.4600826635502966
Epoch  24 ->Error:  0.4608740595702951
Epoch  25 ->Error:  0.4600

In [0]:
def heirarchical(k,noofneurons=[29,14,29],activationFunctions=[None,"linear","linear"],learningRat=0.1,forgraph=False):
    data=pd.read_csv("data.csv").values
    X=data[:,0:29]
    Y=data[:,29]
    
    X = np.array(X, dtype=np.float64)
    X=(X-X.mean(axis=0))/(X.std(axis=0))
    
    numberofLayers=len(noofneurons)
    batchSize=64
    epochs=50
    
    nn=NeuralNet(numberofLayers,noofneurons,activationFunctions)
    nn.trainNetwork(X,None,batchSize,epochs,learningRat)
    nn.batchSize = len(X)
    nn.forwardPropo(X)
    
    dimensions=14
    X=nn.layers[nn.totalLayers-2].activations
    
    aglo = AgglomerativeClustering(linkage="single", n_clusters=k)
    aglo.fit(X)
    labels=aglo.labels_
    labelsDict={}
    for index,label in enumerate(labels):
        labelsDict.setdefault(label,[]).append(index)
    methodPurity=0
    graphLabels=[]
    graphClusters=[]
    for key in labelsDict.keys():
        indexes=labelsDict[key]
        labelss=np.array([Y[index] for index in indexes])
        uniq,counts=np.unique(labelss,return_counts=True)
        countIndex=np.argmax(counts)
        methodPurity+=counts[countIndex]
        if forgraph:
          graphLabels.append(uniq[countIndex]+" : "+str(counts[countIndex]/len(indexes)))
          graphClusters.append(len(indexes))
          continue
        print("========================================")
        print("Cluster: ",key+1)
        print("========================================")
        print("%10s || %2s "%("class","count"))
        print("________________________")
        for a,b in zip(uniq,counts):
          print("%10s || %2s "%(a,b))
        print()
        print("Max class '%s' with purity %s . "%(uniq[countIndex],counts[countIndex]/len(indexes)))
        print("========================================")
    if forgraph:
      return (methodPurity/len(X),graphLabels,graphClusters)
    print("Method clustering Purity= ",methodPurity/len(X))

### Part-4 : Heirarchical Clustering on dimensionality reduced inputs.

#### Part-4-a :Heirarchical Clustering with linear activation functions in autoencoder

In [81]:
heirarchical(5,noofneurons=[29,14,29],activationFunctions=[None,"linear","linear"])

Epoch  0 ->Error:  0.3575779382724174
Epoch  1 ->Error:  0.2787368489133608
Epoch  2 ->Error:  0.24968088446395578
Epoch  3 ->Error:  0.2206209921776696
Epoch  4 ->Error:  0.19033621498761166
Epoch  5 ->Error:  0.16383211019930857
Epoch  6 ->Error:  0.14376443005157227
Epoch  7 ->Error:  0.1297496533894566
Epoch  8 ->Error:  0.12014885734713412
Epoch  9 ->Error:  0.11354311821971032
Epoch  10 ->Error:  0.10890303331821959
Epoch  11 ->Error:  0.1054931531755054
Epoch  12 ->Error:  0.10282013927327624
Epoch  13 ->Error:  0.1005756625781417
Epoch  14 ->Error:  0.09857948023747577
Epoch  15 ->Error:  0.09673385502652314
Epoch  16 ->Error:  0.09499072543578355
Epoch  17 ->Error:  0.09332978875335314
Epoch  18 ->Error:  0.09174484979848262
Epoch  19 ->Error:  0.09023589317850414
Epoch  20 ->Error:  0.0888048217787879
Epoch  21 ->Error:  0.08745338468448512
Epoch  22 ->Error:  0.08618232888681708
Epoch  23 ->Error:  0.08499118794054107
Epoch  24 ->Error:  0.08387837166417134
Epoch  25 ->Error

#### Part-4-b :Heirarchical Clustering with non linear activation functions (ReLu) in autoencoder

In [82]:
heirarchical(5,noofneurons=[29,14,29],activationFunctions=[None,"relu","relu"])

Epoch  0 ->Error:  0.5809002802507022
Epoch  1 ->Error:  0.4911819860151148
Epoch  2 ->Error:  0.4532047369018364
Epoch  3 ->Error:  0.4177582507498161
Epoch  4 ->Error:  0.29126924302300344
Epoch  5 ->Error:  0.20564337368846666
Epoch  6 ->Error:  0.2046655546600491
Epoch  7 ->Error:  0.20587919030327723
Epoch  8 ->Error:  0.20375249269107706
Epoch  9 ->Error:  0.19587987383714517
Epoch  10 ->Error:  0.18365098784119063
Epoch  11 ->Error:  0.16987313502621465
Epoch  12 ->Error:  0.15827141865097322
Epoch  13 ->Error:  0.14961314102730222
Epoch  14 ->Error:  0.14386763479088152
Epoch  15 ->Error:  0.14005744056248018
Epoch  16 ->Error:  0.13767233576894464
Epoch  17 ->Error:  0.13617132709122312
Epoch  18 ->Error:  0.13512869726008397
Epoch  19 ->Error:  0.13438688926405518
Epoch  20 ->Error:  0.1338598485778306
Epoch  21 ->Error:  0.13350093886235892
Epoch  22 ->Error:  0.133170791333968
Epoch  23 ->Error:  0.13067896244995839
Epoch  24 ->Error:  0.12116843717081648
Epoch  25 ->Error:

#### Part-4-c :Heirarchical Clustering with deep autoencoders.

In [83]:
heirarchical(k=5,noofneurons=[29,20,20,14,20,20,29],activationFunctions=[None,"sigmoid","sigmoid","sigmoid","sigmoid","sigmoid","sigmoid"],learningRat=2)

Epoch  0 ->Error:  0.5357232035865315
Epoch  1 ->Error:  0.5321675983245805
Epoch  2 ->Error:  0.5235638180993053
Epoch  3 ->Error:  0.49873941676069355
Epoch  4 ->Error:  0.484855748853895
Epoch  5 ->Error:  0.47493891725856774
Epoch  6 ->Error:  0.4736776902490018
Epoch  7 ->Error:  0.4634237477205377
Epoch  8 ->Error:  0.4625995123110366
Epoch  9 ->Error:  0.4619401248486204
Epoch  10 ->Error:  0.46137024289265255
Epoch  11 ->Error:  0.461621997996031
Epoch  12 ->Error:  0.46144887355621683
Epoch  13 ->Error:  0.46139649646261355
Epoch  14 ->Error:  0.461154442653613
Epoch  15 ->Error:  0.4616892738661498
Epoch  16 ->Error:  0.460237162741095
Epoch  17 ->Error:  0.4608002660323936
Epoch  18 ->Error:  0.4725430914926422
Epoch  19 ->Error:  0.47252309458105957
Epoch  20 ->Error:  0.46344407016882205
Epoch  21 ->Error:  0.46568708362552647
Epoch  22 ->Error:  0.45952723551872093
Epoch  23 ->Error:  0.4607760314553168
Epoch  24 ->Error:  0.459392640353683
Epoch  25 ->Error:  0.458941914